In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from src import RGBNirModule, RGBNirDataModule

dm = RGBNirDataModule(batch_size=512, num_workers=20, pin_memory=True)
dm.setup()

train: 1587395
val: 40080
test: 36421


In [4]:
model = RGBNirModule.load_from_checkpoint('checkpoints/r18rgbnir-val_loss=6.48245-epoch=3.ckpt')
model.hparams

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7feaf04e4310>
Traceback (most recent call last):
  File "/home/juan/miniconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/juan/miniconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1268, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'


In [1]:
from tqdm import tqdm
from src.GLC.metrics import top_30_error_rate
import numpy as np 

model.cuda(1)
dl = dm.val_dataloader()
accs = []
for imgs, labels in tqdm(dl):
    preds = model.predict(imgs)
    accs.append(top_30_error_rate(labels, preds.cpu()))
np.mean(accs)

NameError: name 'model' is not defined

In [5]:
from tqdm import tqdm

model.cuda()
dl = dm.test_dataloader()
labels, observations = [], []
for imgs, observation_ids in tqdm(dl):
    preds = model.predict(imgs)
    values, ixs = preds.topk(30)
    labels += [' '.join([str(i.item()) for i in ix]) for ix in ixs]
    observations += observation_ids

100%|██████████| 72/72 [00:24<00:00,  2.91it/s]


In [10]:
import pandas as pd 

submission = pd.DataFrame({'Id': observations, 'Predicted': labels})
submission.to_csv('submission.csv', index=False)
submission.sample(5)

,Id,Predicted
4656,10062742,815 726 603 566 240 630 637 565 306 106 736 35...
32903,21687924,6152 2524 325 2537 1916 5168 5421 5384 8188 52...
23094,20629603,2196 5045 2617 1047 570 1038 1803 6600 1637 51...
13333,10424541,91 342 21 216 103 207 338 122 333 738 416 587 ...
30863,21472979,7784 5681 5274 2560 6610 1445 4985 7533 2143 7...


In [9]:
sample_submission = pd.read_csv('data/sample_submission.csv')
assert len(sample_submission) == len(submission)
sample_submission.sample(10)

,Id,Predicted
3477,10165186,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
9874,10470993,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
7692,10365076,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
20320,20333104,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
11603,10555567,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
2717,10129049,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
2394,10113571,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
23167,20637647,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
35886,22009824,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
18472,20126089,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
